In [ ]:
import sys

sys.path.append("/kesemi/g2opy/lib/")

import kesemi
import numpy as np

from pathlib import Path

In [ ]:
matcher_type = "ORB"  # OR "SP", "SG"
train_intrinsics = np.asarray([[481.20, 0, 319.50],
                               [0, -480.00, 239.50],
                               [0, 0, 1]])
# Replace if query and training images are taken from different cameras
query_intrinsics = train_intrinsics
depth_scale = 5000

path_to_train_depth_image = Path("test_images/depth/0150.png")
path_to_train_color_image = Path("test_images/rgb/0150.png")
path_to_query_color_image = Path("test_images/rgb/0200.png")

In [ ]:
if matcher_type == "ORB":
    matcher = kesemi.ORB(nfeatures=1000, ratio_threshold=0.7)
elif matcher_type == "SP":
    matcher = kesemi.SuperPoint("superpoint_v1.pth", nn_thresh=0.7)
elif matcher_type == "SG":
    matcher = kesemi.SuperGlue(
        path_to_sp_weights="superpoint_v1.pth",
        path_to_sg_weights="superglue_indoor.pth",
    )
else:
    raise ValueError("Matcher type should be ORB, SuperPoint or SuperGlue")

# On some scenes, the results are better if the threshold is much higher (e.g., 20000)
pose_optimizer = kesemi.PoseOptimizer(query_intrinsics, reproj_tresh=30)

In [ ]:
matches, query_kpts, train_kpts = matcher.match(
    path_to_train_color_image, path_to_query_color_image
)
train_depth_image = kesemi.DepthImage(
    path_to_train_depth_image,
    train_kpts,
    train_intrinsics,
    depth_scale,
)
matches = train_depth_image.filter_matches_with_depth_mask(matches)

kpts_obs = []
keyframe_3d_points = []
for matched_keyframe_kp_i, matched_query_kp_i in matches:
    keyframe_3d_points.append(train_depth_image.kpts_3d[matched_keyframe_kp_i])
    kpts_obs.append(query_kpts[matched_query_kp_i])
T = pose_optimizer.optimize(keyframe_3d_points, kpts_obs)

In [ ]:
np.savetxt("T.txt", T)